# Importing

In [34]:
import pandas as pd
import numpy as np
import math

# Reading data

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
train_data = pd.read_csv('kc_house_train_data.csv',dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

# Converting to numpy function

In [15]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’

    # this will convert the features into a numpy matrix:
    features_matrix = data[features].to_numpy()
    # assign the column of data associated with the target to the variable ‘output_array’
    output_array = data[output].to_numpy()
    return features_matrix, output_array 


# Finding predictions functions

In [14]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions


# Finding derivative function

In [49]:
def feature_derivative(errors, feature):
    derivative = -2*np.dot(feature, errors)
    return derivative 

# Finding weights function

In [63]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = predict_outcome(feature_matrix, weights)
        # compute the errors as predictions - output:
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative_i = feature_derivative(errors, feature_matrix[:,i])
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares = gradient_sum_squares + derivative_i**2
            # update the weight based on step size and derivative:
            weights[i] = weights[i]- (step_size*derivative_i)
        gradient_magnitude = math.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return weights


# Finding RSS function

In [71]:
def get_residual_sum_of_squares(output, predictions):
    RSS = sum((output-predictions)**2)
    return RSS 

# finding weights for model 1

In [64]:
simple_features = ['sqft_living']
my_output= 'price'
simple_feature_matrix, output = get_numpy_data(train_data, simple_features, my_output)
initial_weights = [-47000., 1.]
step_size = 7e-12
tolerance = 2.5e7

In [66]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,tolerance)
simple_weights

array([-46999.88716555,    281.91211918])

In [55]:
simple_weights

array([-46999.88716555,    281.91211918])

# Quiz Question: 

**What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

***281.9***



# predicting house prices on test data, model 1

In [68]:
simple_features = ['sqft_living']
my_output= 'price'
test_simple_feature_matrix, test_output = get_numpy_data(test_data, simple_features, my_output)

In [70]:
test_predictions = predict_outcome(test_simple_feature_matrix, simple_weights)
test_predictions

array([356134.443255  , 784640.86440132, 435069.83662406, ...,
       663418.65315598, 604217.10812919, 240550.47439317])

# Quiz Question:

**What is the predicted price for the 1st house in the Test data set for model 1 (round to nearest dollar)?**

***356134***



# RSS of model 1 all test data


In [81]:
RSS1_test = get_residual_sum_of_squares(test_output,test_predictions)
"{:e}".format(RSS1_test)

'2.754000e+14'

# finding weights for model 2

In [74]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
feature_matrix, output = get_numpy_data(train_data, model_features,my_output)
initial_weights = [-100000., 1., 1.]
step_size = 4e-12
tolerance = 1e9

In [75]:
model2_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size,tolerance)
model2_weights

array([-9.99999688e+04,  2.45072603e+02,  6.52795267e+01])

# predicting house prices on test data, model 2

In [77]:
test_feature_matrix, test_output = get_numpy_data(test_data, model_features, my_output)
test_predictions_model2 = predict_outcome(test_feature_matrix, model2_weights)
test_predictions_model2

array([366651.41162949, 762662.39850726, 386312.09557541, ...,
       682087.39916306, 585579.27901327, 216559.20391786])

# Quiz Question: 

**What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

***366651***


# question

**What is the actual price for the 1st house in the Test data set?**



In [78]:
test_output[0]

310000.0

# Quiz Question: 

**Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**


***model 1***



# RSS of model 2 all test data


In [80]:
RSS2_test = get_residual_sum_of_squares(test_output,test_predictions_model2)
"{:e}".format(RSS2_test)

'2.702634e+14'

# Quiz Question: 

**Which model (1 or 2) has lowest RSS on all of the TEST data?**

***model 2***